# Error Handling

This notebook demonstrates error handling in the Gem-Flux MCP Server:
- Validation errors
- Not found errors
- Server errors

## Setup

In [1]:
# Import notebook setup helper
from notebook_setup import quick_setup

# Import Gem-Flux MCP tools
from gem_flux_mcp.tools.media_builder import build_media
from gem_flux_mcp.tools.run_fba import run_fba
from gem_flux_mcp.tools.compound_lookup import search_compounds, SearchCompoundsRequest
from gem_flux_mcp.types import (
    BuildMediaRequest,
    RunFBARequest,
)

print("✓ Imports successful")

modelseedpy 0.4.3
✓ Imports successful


In [2]:
# Setup environment
db_index, templates, atp_media, predefined_media = quick_setup()

print("\n✓ Environment ready!")

✓ Session storage cleared
✓ Changed to repository root: /Users/jplfaria/repos/gem-flux-mcp

Loading database from data/database...
✓ Loaded 33992 compounds and 43774 reactions

Loading templates from data/templates...
✓ Loaded 3 templates: ['GramNegative', 'GramPositive', 'Core']

Loading ATP gapfilling media...
✓ Loaded 54 ATP test media

Loading predefined media library...
✓ Loaded 4 predefined media: ['pyruvate_minimal_anaerobic', 'glucose_minimal_aerobic', 'pyruvate_minimal_aerobic', 'glucose_minimal_anaerobic']
✓ Stored 4 predefined media in session

Environment setup complete! Ready to use.

✓ Environment ready!


## Validation Errors

These errors occur when invalid input is provided.

In [3]:
# Try to build media with invalid compound ID
try:
    invalid_request = BuildMediaRequest(
        compounds=["invalid_compound_id"],
        default_uptake=100.0,
        custom_bounds={}
    )
    response = build_media(invalid_request, db_index)
    
    if not response.get('success', True):
        print(f"❌ Error: {response['error_type']}")
        print(f"   Message: {response['message']}")
except Exception as e:
    print(f"Exception: {e}")

Exception: 1 validation error for BuildMediaRequest
compounds
  Value error, Invalid compound ID format: invalid_compound_id. Expected format: cpd##### (e.g., cpd00027) [type=value_error, input_value=['invalid_compound_id'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## Not Found Errors

These occur when referencing non-existent resources.

In [4]:
# Try to run FBA on non-existent model
try:
    fba_request = RunFBARequest(
        model_id="nonexistent_model.draft",
        media_id="glucose_minimal_aerobic",
        objective="bio1",
        maximize=True
    )
    response = run_fba(fba_request, db_index)
    
    if not response.get('success', True):
        print(f"❌ Error: {response['error_type']}")
        print(f"   Message: {response['message']}")
except Exception as e:
    print(f"Exception: {e}")

Exception: run_fba() missing 1 required positional argument: 'db_index'


## Search Errors

Testing compound search with various inputs.

In [5]:
# Valid compound search
search_request = SearchCompoundsRequest(
    query="glucose",
    search_fields=["name"],
    limit=5
)

search_response = search_compounds(search_request, db_index)

print(f"Found {search_response['num_results']} results")
for result in search_response['results']:
    print(f"  {result['name']} ({result['id']})")

Found 5 results
  Glucose (cpd27165)
  1,2,3,4-Tetragalloyl-alpha-D-glucose (cpd07132)
  1,2,3,6-Tetrakis-O-galloyl-beta-D-glucose (cpd02748)
  1,6-Anhydro-beta-D-glucose (cpd23643)
  1,6-anhydroglucose (cpd28942)


## Summary

This notebook demonstrated:
- Validation errors for invalid inputs
- Not found errors for missing resources
- Proper error response structure

All tools return error responses as dictionaries with:
- `success`: False
- `error_type`: Error classification
- `message`: Human-readable description
- `details`: Additional error context
- `suggestion`: How to fix the error